In [1]:
import pandas as pd
from graphviz import Digraph
df = pd.read_excel('SQL取数.xlsx')
edges_df = pd.read_excel('节点.xlsx')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
df = df.append({'metric_name': '1-CURR', 'metric_value': 100 - df.loc[df['metric_name'] == 'CURR', 'metric_value'].values[0]}, ignore_index=True)
df = df.append({'metric_name': '1-RURR', 'metric_value': 100 - df.loc[df['metric_name'] == 'RURR', 'metric_value'].values[0]}, ignore_index=True)
df = df.append({'metric_name': '1-SURR', 'metric_value': 100 - df.loc[df['metric_name'] == 'SURR', 'metric_value'].values[0]}, ignore_index=True)
df = df.append({'metric_name': '1-NURR', 'metric_value': 100 - df.loc[df['metric_name'] == 'NURR', 'metric_value'].values[0]}, ignore_index=True)

In [3]:
replace_dict = {
    'current_user': '当前用户',
    'reactivated_user': '重新激活用户',
    'resurrected_user': '复活用户',
    'new_user': '新用户',
    'at_risk_waus': '周活跃用户流失风险',
    'at_risk_maus': '月活跃用户流失风险',
    'dead_users': '流失用户'
}

# 使用replace方法进行替换
df['metric_name'] = df['metric_name'].replace(replace_dict)

In [4]:
nodes_df = df[df['metric_name'].isin(['当前用户','重新激活用户','复活用户','新用户','周活跃用户流失风险','月活跃用户流失风险','流失用户'])]

In [5]:
nodes_df.loc[nodes_df['metric_name']=='当前用户', '备注'] = '(今日活跃且过去7天内至少活跃1次)'
nodes_df.loc[nodes_df['metric_name']=='重新激活用户', '备注'] = '(首次回访)'
nodes_df.loc[nodes_df['metric_name']=='复活用户', '备注'] = '(首次回访)'
nodes_df.loc[nodes_df['metric_name']=='新用户', '备注'] = '(首次使用应用)'
nodes_df.loc[nodes_df['metric_name']=='周活跃用户流失风险', '备注'] = '(今日不活跃，但过去1-6天活跃)'
nodes_df.loc[nodes_df['metric_name']=='月活跃用户流失风险', '备注'] = '(今日不活跃，过去1-6天不活跃，但过去7-29天活跃)'
nodes_df.loc[nodes_df['metric_name']=='流失用户', '备注'] = '(今日及过去29天不活跃)'

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [6]:
nodes_df.rename(columns={'metric_name':'节点','metric_value':'数据'},inplace=True)
nodes_df['数据']=nodes_df['数据'].astype(int)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/dt/gvr9w7tj7zjbg79hbttt_ws40000gn/T/ipykernel_54496/2119995021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df['数据']=nodes_df['数据'].astype(int)


In [7]:
edges_df = edges_df.merge(df,how='left',on='metric_name')
edges_df['指标'] = edges_df['metric_name'].astype(str)+':'+edges_df['metric_value'].astype(str)+'%'

In [8]:
dot = Digraph(comment='用户活跃度分析', engine='dot')
#dot.attr(splines='ortho')  # 设置连接线为直线
# 定义节点颜色（可选）
node_colors = {
    '重新激活用户': 'lightgreen',
    '新用户': 'lightgreen',
    '复活用户': 'lightgreen',
    '当前用户': 'lightblue',
    '周活跃用户流失风险': 'lightcoral',
    '月活跃用户流失风险': 'lightsalmon',
    '流失用户': 'lavender'
}

# 创建子图以固定节点位置
with dot.subgraph() as s:
    s.attr(rank='same')
    for node in ['重新激活用户', '新用户', '复活用户']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)

with dot.subgraph() as s:
    s.attr(rank='same')
    for node in ['当前用户']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)

with dot.subgraph() as s:
    s.attr(rank='same')
    for node in ['周活跃用户流失风险']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)

with dot.subgraph() as s:
    s.attr(rank='same')
    for node in ['月活跃用户流失风险']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)

with dot.subgraph() as s:
    s.attr(rank='same')
    for node in ['流失用户']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)
    
# 添加带有标签和颜色的边
for index, row in edges_df.iterrows():
    end_node_color = node_colors.get(row['终点'], 'black')  # 获取终点节点的颜色
    dot.edge(row['起点'], row['终点'], label=row['指标'], color=end_node_color)

# 保存图
dot.format = 'png'
dot.render('/Volumes/文枢工作空间/资产配置/新开城旧车交付/复盘/user_activity_analysis.png', view=True)

'/Volumes/文枢工作空间/资产配置/新开城旧车交付/复盘/user_activity_analysis.png.png'

In [9]:
node = '重新激活用户'
label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"

In [10]:
nodes_df

,节点,数据,备注
0,当前用户,12538,(今日活跃且过去7天内至少活跃1次)
1,重新激活用户,5034,(首次回访)
2,复活用户,2030,(首次回访)
3,新用户,4253,(首次使用应用)
4,周活跃用户流失风险,52719,(今日不活跃，但过去1-6天活跃)
5,月活跃用户流失风险,137192,(今日不活跃，过去1-6天不活跃，但过去7-29天活跃)
6,流失用户,277857,(今日及过去29天不活跃)


In [11]:
edges_df['指标'] = edges_df['metric_name'].astype(str)+':'+edges_df['metric_value'].astype(str)+'%'

In [12]:
for node in ['重新激活用户', '新用户', '复活用户']:
        label = f"{node}\n{nodes_df[nodes_df['节点'] == node]['备注'].values[0]}\n{nodes_df[nodes_df['节点'] == node]['数据'].values[0]}"
        color = node_colors.get(node, 'white')
        s.node(node, label, shape='box', style='filled', fillcolor=color)